In [0]:
# Turning warning messages off
options(warn = 0) # i don't care for the messages

# Libraries
library(dataiku)
library(rpart)
library(dplyr)
library(caret)
library(pROC) # For AUC calculation
library(data.table)
library(mlflow)
library(reticulate)
library(Matrix)

In [0]:
# Recipe inputs
df_base_train <- dkuReadDataset("base_train", samplingMethod="head", nbRows=100000)
df_base_validation <- dkuReadDataset("base_validation", samplingMethod="head", nbRows=100000)

In [0]:
# Training track_min_dist ~ island_groups
# we will need to also include island_groups 
# in the final outcome prediction model to adjust for the confounding

base_track_model  <- rpart(track_min_dist  ~ island_groups,
                          data = df_base_train,
                          method = "anova")

In [0]:
# Training structural equation for wind speed
# wind_speed = f(track_min_dist, eps)


base_wind_model <- rpart(wind_max ~ track_min_dist,
                       data = df_base_train,
                       method = "anova")

In [0]:
# Training structural equation for rain speed
# rain_total = f(track_min_dist, eps)

base_rain_model <- rpart(rain_total ~ track_min_dist,
                       data = df_base_train,
                       method = "anova")

In [0]:
# Adding the predicted parents' to the training dataset

df_base_train <- df_base_train %>%
  mutate(track_min_dist_pred = predict(base_track_model, newdata = df_base_train), # predicted min_dist
         wind_max_pred = predict(base_wind_model, newdata = df_base_train),
         rain_total_pred = predict(base_rain_model, newdata = df_base_train), # Updating interaction terms
         wind_blue_ss = wind_max_pred * blue_ss_frac,
         wind_yellow_ss = wind_max_pred * yellow_ss_frac,
         wind_orange_ss = wind_max_pred * orange_ss_frac,
         wind_red_ss = wind_max_pred * red_ss_frac,
         rain_blue_ss = rain_total_pred * blue_ls_frac,
         rain_yellow_ss = rain_total_pred * yellow_ls_frac,
         rain_orange_ss = rain_total_pred * orange_ls_frac,
         rain_red_ss = rain_total_pred * red_ls_frac,
         )

In [0]:
# Adding the predicted parents' to the validation dataset
# predicting for wind and rainfall for the validation dataset
df_base_validation <- df_base_validation %>%
  mutate(track_min_dist_pred = predict(base_track_model, newdata = df_base_validation),  # First predict for track_min_dist from regions
    wind_max_pred = predict(base_wind_model, newdata = df_base_validation),
    rain_total_pred = predict(base_rain_model, newdata = df_base_validation),
    wind_blue_ss = wind_max_pred * blue_ss_frac,
    wind_yellow_ss = wind_max_pred * yellow_ss_frac,
    wind_orange_ss = wind_max_pred * orange_ss_frac,
    wind_red_ss = wind_max_pred * red_ss_frac,
    rain_blue_ss = rain_total_pred * blue_ls_frac,
    rain_yellow_ss = rain_total_pred * yellow_ls_frac,
    rain_orange_ss = rain_total_pred * orange_ls_frac,
    rain_red_ss = rain_total_pred * red_ls_frac,
  )


In [0]:
# Parameter tuning

# Define tuning grid
tune_grid <- expand.grid(
  nrounds = c(50, 100, 150),       # Number of boosting rounds
  max_depth = c(3, 6, 9),          # Maximum tree depth
  eta = c(0.01, 0.1, 0.3),         # Learning rate
  gamma = 0,                       # Minimum loss reduction
  colsample_bytree = 0.8,          # Feature selection rate
  min_child_weight = 1,            # Minimum instance weight
  subsample = 0.8                  # Sample ratio per boosting round
)


# Create an empty list to store results
results_list <- list()

# Extra data prep
# Ensure target variable is a factor for classification
df_base_train$damage_binary <- as.factor(df_base_train$damage_binary)
df_base_validation$damage_binary <- as.factor(df_base_validation$damage_binary)

# Train the model using manual grid search
grid_id <- 1  # Index for list storage

# Iterate over all combinations of hyperparameters
for (i in 1:nrow(tune_grid)) {
  params <- tune_grid[i, ]
       
        # setting seed for reproducibility
        set.seed(1234)
        # Train the model with specific hyperparameters
        xgb_model <- train(
          as.factor(damage_binary) ~ wind_max_pred +
            rain_total_pred +
            roof_strong_wall_strong +
            roof_strong_wall_light +
            roof_strong_wall_salv +
            roof_light_wall_strong +
            roof_light_wall_light +
            roof_light_wall_salv +
            roof_salv_wall_strong +
            roof_salv_wall_light +
            roof_salv_wall_salv +
            ls_risk_pct +
            ss_risk_pct +
            wind_blue_ss +
            wind_yellow_ss +
            wind_orange_ss +
            wind_red_ss +
            rain_blue_ss +
            rain_yellow_ss +
            rain_orange_ss +
            rain_red_ss +
            island_groups, # CONFOUNDER ADJUSTED
          data = df_base_train,
          method = "xgbTree", # XGBoost method
          trControl = trainControl(method = "none"),  # No automatic validation
          tuneGrid = params # Hyperparameter grid
        )

        # Make probability predictions for classification
        val_predictions <- predict(xgb_model, newdata = df_base_validation, type = "prob")[,2]  # Probability of class 1

        # Compute AUC (better for classification)
        auc_value <- auc(df_base_validation$damage_binary, val_predictions)

        # Store results efficiently in a list
        results_list[[i]] <- data.frame(params, AUC = auc_value)
}

# Convert list to data frame
results <- rbindlist(results_list)

# Print the best hyperparameter combination (highest AUC)
best_params <- results[which.max(results$AUC), ]
print(best_params)

In [0]:
best_params

In [0]:
# Training based on tuned parameters

# Combine Training and Validation datasets for final training

final_training_df  <- rbind(df_base_train,
                           df_base_validation)


# Extract the best parameters (remove AUC column)
best_params_model <- best_params %>% # Remove AUC column if present
    select(-AUC)

damage_fit_class_full <- xgb_model <- train(
          as.factor(damage_binary) ~ wind_max_pred +
            rain_total_pred +
            roof_strong_wall_strong +
            roof_strong_wall_light +
            roof_strong_wall_salv +
            roof_light_wall_strong +
            roof_light_wall_light +
            roof_light_wall_salv +
            roof_salv_wall_strong +
            roof_salv_wall_light +
            roof_salv_wall_salv +
            ls_risk_pct +
            ss_risk_pct +
            wind_blue_ss +
            wind_yellow_ss +
            wind_orange_ss +
            wind_red_ss +
            rain_blue_ss +
            rain_yellow_ss +
            rain_orange_ss +
            rain_red_ss +
            island_groups,
          data = final_training_df, # USE TRAINING AND VALIDATION SETS COMBINED
          method = "xgbTree", # XGBoost method
          trControl = trainControl(method = "none"),  # No automatic validation
          tuneGrid = best_params_model # USE BEST PARAMETER
        )

In [0]:
# Sanity Check
# testing on the training datasets (training + validation)

## Outcome prediction on the final_training_df dataset
## default function predict returns class probabilities (has two columns)
y_pred <- predict(damage_fit_class_full,
                  newdata = final_training_df)

In [0]:
# using table function
conf_matrix <- confusionMatrix(y_pred,
                     final_training_df$damage_binary,  
                     positive = "1"
                     )
conf_matrix

In [0]:
accuracy <- conf_matrix$overall['Accuracy']

cat("test-set accuracy of minimal SCM model:", accuracy, sep = " ")

In [0]:
# Logging the model and parameter using MLflow

# set tracking URI
mlflow_set_tracking_uri("http://127.0.0.1:5000")

# Ensure any active run is ended
suppressWarnings(try(mlflow_end_run(), silent = TRUE))

# set experiment
# Logging metrics for model training and the parameters used
mlflow_set_experiment(experiment_name = "SCM - XGBOOST classification (Training metircs)")

# Start MLflow run
mlflow_start_run(nested = FALSE)

# Ensure the run ends even if an error occurs
on.exit(mlflow_end_run(), add = TRUE)

# Extract the best parameters (remove AUC column)
best_params_model <- best_params %>% # Remove AUC column if present
    select(-AUC)

damage_fit_class_full <- xgb_model <- train(
          as.factor(damage_binary) ~ wind_max_pred +
            rain_total_pred +
            roof_strong_wall_strong +
            roof_strong_wall_light +
            roof_strong_wall_salv +
            roof_light_wall_strong +
            roof_light_wall_light +
            roof_light_wall_salv +
            roof_salv_wall_strong +
            roof_salv_wall_light +
            roof_salv_wall_salv +
            ls_risk_pct +
            ss_risk_pct +
            wind_blue_ss +
            wind_yellow_ss +
            wind_orange_ss +
            wind_red_ss +
            rain_blue_ss +
            rain_yellow_ss +
            rain_orange_ss +
            rain_red_ss +
            island_groups,
          data = final_training_df, # USE TRAINING AND VALIDATION SETS COMBINED
          method = "xgbTree", # XGBoost method
          trControl = trainControl(method = "none"),  # No automatic validation
          tuneGrid = best_params_model # USE BEST PARAMETER
        )


# summarize results
conf_matrix <- confusionMatrix(y_pred,
                     final_training_df$damage_binary,  
                     positive = "1"
                     )

# accuracy
accuracy  <- conf_matrix$overall['Accuracy']

# Positive class = 1, precision, recall, and F1
# Extract precision, recall, and F1 score
precision <- conf_matrix$byClass['Precision']
recall <- conf_matrix$byClass['Recall']
f1_score <- conf_matrix$byClass['F1']


# Log parameters and metrics
mlflow_log_param("model_type", "scm-xgboost-classification")
mlflow_log_metric("accuracy", accuracy)
mlflow_log_metric("F1", F1)
mlflow_log_metric("Precision", precision)
mlflow_log_metric("Recall", recall)


# Save model
#saveRDS(model, file = file.path(path_2_folder, "spam_clas_model.rds"))

# End MLflow run
mlflow_end_run()

In [0]:
# Recipe outputs
managed_folder_path <- dkuManagedFolderPath("xcPrnvPS")

saveRDS(damage_fit_class_full, file = paste0(managed_folder_path, "/base_clas_full_model.rds"))

saveRDS(base_wind_model, file = paste0(managed_folder_path, "/base_wind_model.rds"))

saveRDS(base_rain_model, file = paste0(managed_folder_path, "/base_rain_model.rds"))
saveRDS(base_track_model, file = paste0(managed_folder_path, "/base_track_model.rds"))